# 8 puzzle solver
### This project can solve the 8 puzzle using 3 different methods:
* Breadth First Search (BFS)
* Depth First Search
* A start (ast)
After the chosen algorithm is run the set of moves to reach the solution from the current state is printed. The time of execution is also printed.

In [1]:
import math
import time
import psutil
import os

Board = [[0 for x in range(3)] for y in range(3)]
solvedBoard = [['0','1','2'],['3','4','5'],['6','7','8']]

def positionOfZero(boardState):
	for x in range(0,3):
		for y in range(0,3):
			if boardState[x][y] is "0":
				answer = [0 for z in range(2)]
				answer[0] = x
				answer[1] = y
				return answer

def possibleMoves(boardState):
	zero = positionOfZero(boardState)
	answer = []
	if zero[0] == 0:
		answer.append("Down")
	if zero[0] == 2:
		answer.append("Up")
	if zero[0] == 1:
		answer.append("Up")
		answer.append("Down")
	if zero[1] == 0:
		answer.append("Right")
	if zero[1] == 2:
		answer.append("Left")
	if zero[1] == 1:
		answer.append("Left")
		answer.append("Right")
	return answer

def changeBoard(boardState,move):
	zero = positionOfZero(boardState)
	if move is "Left":
		boardState[zero[0]][zero[1]] = boardState[zero[0]][zero[1]-1]
		boardState[zero[0]][zero[1]-1] = "0"
	if move is "Right":
		boardState[zero[0]][zero[1]] = boardState[zero[0]][zero[1]+1]
		boardState[zero[0]][zero[1]+1] = "0"
	if move is "Up":
		boardState[zero[0]][zero[1]] = boardState[zero[0]-1][zero[1]]
		boardState[zero[0]-1][zero[1]] = "0"
	if move is "Down":
		boardState[zero[0]][zero[1]] = boardState[zero[0]+1][zero[1]]
		boardState[zero[0]+1][zero[1]] = "0"
	return boardState;

def copy(boardState):
	board2 = [[0 for x in range(3)] for y in range(3)]
	for x in range(0,3):
		for y in range(0,3):
			board2[x][y] = boardState[x][y]
	return board2

def getChildren(boardState):
	children = []
	possible = possibleMoves(boardState)
	for x in range(len(possible)):
		children.append(node(None,None,None,possible[x],changeBoard(copy(boardState),possible[x])))
	return children

def priority(boardState):
	sum = 0
	for x in range(0,3):
		for y in range(0,3):
			sum += differenceFromCurrent(x,y,boardState[x][y])
	return sum

def priority2(move):
	if move is "Up":
		return 3
	if move is "Down":
		return 2
	if move is "Left":
		return 1
	if move is "Right":
		return 0

def differenceFromCurrent(x,y,num):
	originalX = None
	originalY = None
	if num is '0':
		return 0
	if num is '1':
		originalX = 0
		originalY = 1
	if num is '2':
		originalX = 0
		originalY = 2
	if num is '3':
		originalX = 1
		originalY = 0
	if num is '4':
		originalX = 1
		originalY = 1
	if num is '5':
		originalX = 1
		originalY = 2
	if num is '6':
		originalX = 2
		originalY = 0
	if num is '7':
		originalX = 2
		originalY = 1
	if num is '8':
		originalX = 2
		originalY = 2
	return abs(originalX-x)+abs(originalY-y)


class node:
	__childs = None
	__parent = None
	__nodeBoardState = None
	__moveFromPrevious = None
	__key = ""
	__depth = None
	def __init__(self, childs, parent, depth, moveFromPrevious, boardState):
		self.__childs = childs
		self.__parent = parent
		self.__moveFromPrevious = moveFromPrevious
		self.__nodeBoardState = boardState
		self.__depth = depth
		for x in range(0,3):
			for y in range(0,3):
				self.__key += boardState[x][y]+""

	def getBoard(self):
		return self.__nodeBoardState

	def getChildren(self):
		return self.__childs

	def getParent(self):
		return self.__parent

	def setParent(self, parent):
		self.__parent = parent

	def setChildren(self, childs):
		self.__childs = childs

	def getMoveFromPrevious(self):
		return self.__moveFromPrevious

	def getKey(self):
		return self.__key

	def getDepth(self):
		return self.__depth

	def setDepth(self, depth):
		self.__depth = depth

def initialize():
	for x in range(0,3):
		for y in range(0,3):
			Board[x][y] = board[3*x+y]

def print_output(steps,pathCost,count,maxDepth,timeStart):
	print("path_to_goal: "+("['"+"', '".join(steps)+"']")+"\n")
	print("cost_of_path: "+str(pathCost)+"\n")
	print("nodes_expanded: "+str(count)+"\n")
	print("search_depth: "+str(pathCost)+"\n")
	print("max_search_depth: "+str(maxDepth)+"\n")
	
	#ramUsage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss /1000.0
	process = psutil.Process(os.getpid())
	ramUsage = process.memory_info().rss / (1024*1024)
	timeElapsed = time.time()-timeStart
	print("running_time: "+str(timeElapsed)+" seconds \n")
	print("max_ram_usage: "+str(ramUsage)+" MB \n")

def solver():
    initialize()
    if method == "bfs":
        timeStart = time.time()
        startNode = node(None,None,0,None,Board)
        queue = [startNode]
        visited = set([])
        visited.add(startNode.getKey())
        current = queue[0]
        count = 0
        maxDepth = 0
        while current.getBoard()!=solvedBoard:
            current.setChildren(getChildren(current.getBoard()))
            toAdd = current.getChildren()
            for z in range(0,len(toAdd)):
                if not(toAdd[z].getKey() in visited):
                    toAdd[z].setParent(current)
                    toAdd[z].setDepth(current.getDepth()+1)
                    if toAdd[z].getDepth() > maxDepth:
                        maxDepth = toAdd[z].getDepth()
                    queue.append(toAdd[z])
                    visited.add(toAdd[z].getKey())
            queue.remove(current)
            count += 1
            current = queue[0]
        steps = []
        pathCost = current.getDepth()
        while current.getParent() != None:
            steps.append(current.getMoveFromPrevious())
            current = current.getParent()
        steps.reverse()
        print_output(steps,pathCost,count,maxDepth,timeStart)


    if method == "dfs":
        timeStart = time.time()
        startNode = node(None,None,0,None,Board)
        queue = [startNode]
        visited = set([])
        visited.add(startNode.getKey())
        current = queue[0]
        count = 0
        maxDepth = 0
        while current.getBoard()!=solvedBoard:
            current.setChildren(getChildren(current.getBoard()))
            toAdd = current.getChildren()
            toAdd.reverse()
            added = 0
            for z in range(0,len(toAdd)):
                if not(toAdd[z].getKey() in visited):
                    toAdd[z].setParent(current)
                    toAdd[z].setDepth(current.getDepth()+1)
                    if toAdd[z].getDepth() > maxDepth:
                        maxDepth = toAdd[z].getDepth()
                    queue.append(toAdd[z])
                    added += 1
                    visited.add(toAdd[z].getKey())

            queue.pop(len(queue)-1-added)
            count += 1
            current = queue[len(queue)-1]
        steps = []
        pathCost = current.getDepth()
        while current.getParent() != None:
            steps.append(current.getMoveFromPrevious())
            current = current.getParent()
        steps.reverse()
        print_output(steps,pathCost,count,maxDepth,timeStart)


    if method == "ast":
        timeStart = time.time()
        startNode = node(None,None,0,None,Board)
        queue = [startNode]
        visited = set([])
        visited.add(startNode.getKey())
        priorities = [priority(startNode.getBoard())]
        current = queue[0]
        count = 0
        maxDepth = 0
        while current.getBoard()!=solvedBoard:
            current.setChildren(getChildren(current.getBoard()))
            toAdd = current.getChildren()
            for z in range(0,len(toAdd)):
                if not(toAdd[z].getKey() in visited):
                    toAdd[z].setParent(current)
                    toAdd[z].setDepth(current.getDepth()+1)
                    if toAdd[z].getDepth() > maxDepth:
                        maxDepth = toAdd[z].getDepth()
                    cost = priority(toAdd[z].getBoard())+toAdd[z].getDepth()
                    priorities.append(cost)
                    queue.append(toAdd[z])
                    visited.add(toAdd[z].getKey())

            queue.remove(current)
            priorities.remove(priority(current.getBoard())+current.getDepth())
            count += 1
            currentTempPriority = 100
            toUseIndex = 0
            for y in range(0,len(priorities)):
                if priorities[y] < currentTempPriority:
                    toUseIndex = y
                    currentTempPriority = priorities[y]
            current = queue[toUseIndex]
        steps = []
        pathCost = current.getDepth()
        while current.getParent() != None:
            steps.append(current.getMoveFromPrevious())
            current = current.getParent()
        steps.reverse()
        print_output(steps,pathCost,count,maxDepth,timeStart)

You can choose any of the three methods mentioned (bfs,dfs,ast). Then, input the board as a string by reading each row as a string and concatenating all of the strings for all of the rows into one string. (ex: a solved board is "0,1,2,3,4,5,6,7,8")

In [2]:
method = "ast"
boardStr = "1,2,0,3,4,5,6,7,8"
board = boardStr.split(",")
solver()

path_to_goal: ['Left', 'Left']

cost_of_path: 2

nodes_expanded: 2

search_depth: 2

max_search_depth: 2

running_time: 0.0009992122650146484 seconds 

max_ram_usage: 48.21875 MB 

